In [1]:
# ループ学習した各iterationでのレーティングバトルを実行
# tag_prefix = "rl_loop_200815_1_"

In [3]:
from bson import ObjectId
import numpy as np
from pokeai.util import json_load, pickle_load, ROOT_DIR, DATASET_DIR, yaml_dump
from collections import Counter
import json
import yaml
import os
import copy

In [2]:
file_base_dir = r"D:\dev\pokeai\pokeai\experiment\gmm\rl\rl_loop_200815"
tag_prefix = "rl_loop_200815_1_"

In [4]:
with open(os.path.join(file_base_dir, f"{tag_prefix}run.bat"), "r") as f:
    bat_lines = f.readlines()

In [5]:
bat_lines

['\n',
 'python -m pokeai.ai.generate_party rl_loop_200815_1_hc_0 -n 871 -r finalgoodmove1vs1\n',
 '\n',
 'python -m pokeai.ai.generate_party rl_loop_200815_1_allonce_0 --all_pokemon_once -r finalgoodmove1vs1\n',
 'python -m pokeai.ai.generic_move_model.rl_train D:\\dev\\pokeai\\pokeai\\experiment\\gmm\\rl\\rl_loop_200815\\rl_loop_200815_1_0.yaml --trainer_id 5f37a6829949403bf6af940b \n',
 'python -m pokeai.ai.party_feature.hillclimb_party 5f37a6829949403bf6af940b rl_loop_200815_1_hc_1 -r finalgoodmove1vs1 -n 871 --generations 100 --populations 10 --party_feature_penalty 1.0\n',
 'rem {"iternum": 0, "trainer_id": "5f37a6829949403bf6af940b", "train_party_tags": ["rl_loop_200815_1_allonce_0", "rl_loop_200815_1_hc_0"], "gen_party_tag": "rl_loop_200815_1_hc_1"}\n',
 '\n',
 'python -m pokeai.ai.generate_party rl_loop_200815_1_allonce_1 --all_pokemon_once -r finalgoodmove1vs1\n',
 'python -m pokeai.ai.generic_move_model.rl_train D:\\dev\\pokeai\\pokeai\\experiment\\gmm\\rl\\rl_loop_200815\\r

In [6]:
configs = []
for line in bat_lines:
    if line.startswith("rem "):
        configs.append(json.loads(line[4:]))

In [7]:
configs

[{'iternum': 0,
  'trainer_id': '5f37a6829949403bf6af940b',
  'train_party_tags': ['rl_loop_200815_1_allonce_0', 'rl_loop_200815_1_hc_0'],
  'gen_party_tag': 'rl_loop_200815_1_hc_1'},
 {'iternum': 1,
  'trainer_id': '5f37a6829949403bf6af940c',
  'train_party_tags': ['rl_loop_200815_1_allonce_1', 'rl_loop_200815_1_hc_1'],
  'gen_party_tag': 'rl_loop_200815_1_hc_2'},
 {'iternum': 2,
  'trainer_id': '5f37a6829949403bf6af940d',
  'train_party_tags': ['rl_loop_200815_1_allonce_2', 'rl_loop_200815_1_hc_2'],
  'gen_party_tag': 'rl_loop_200815_1_hc_3'},
 {'iternum': 3,
  'trainer_id': '5f37a6829949403bf6af940e',
  'train_party_tags': ['rl_loop_200815_1_allonce_3', 'rl_loop_200815_1_hc_3'],
  'gen_party_tag': 'rl_loop_200815_1_hc_4'},
 {'iternum': 4,
  'trainer_id': '5f37a6829949403bf6af940f',
  'train_party_tags': ['rl_loop_200815_1_allonce_4', 'rl_loop_200815_1_hc_4'],
  'gen_party_tag': 'rl_loop_200815_1_hc_5'},
 {'iternum': 5,
  'trainer_id': '5f37a6829949403bf6af9410',
  'train_party_tags'

In [8]:
rate_ids = [ObjectId() for _ in range(len(configs))]

In [9]:
import subprocess

In [14]:
for config, rate_id in zip(configs, rate_ids):
    cmd = f"python -m pokeai.ai.generic_move_model.rl_rating_battle {config['trainer_id']} {','.join(config['train_party_tags'])} --rate_id {rate_id}"
    print(cmd)
    subprocess.check_call(cmd, shell=True)

python -m pokeai.ai.generic_move_model.rl_rating_battle 5f37a6829949403bf6af940b rl_loop_200815_1_allonce_0,rl_loop_200815_1_hc_0 --rate_id 5f478d7f60f6bad00ef43ff8
python -m pokeai.ai.generic_move_model.rl_rating_battle 5f37a6829949403bf6af940c rl_loop_200815_1_allonce_1,rl_loop_200815_1_hc_1 --rate_id 5f478d7f60f6bad00ef43ff9
python -m pokeai.ai.generic_move_model.rl_rating_battle 5f37a6829949403bf6af940d rl_loop_200815_1_allonce_2,rl_loop_200815_1_hc_2 --rate_id 5f478d7f60f6bad00ef43ffa
python -m pokeai.ai.generic_move_model.rl_rating_battle 5f37a6829949403bf6af940e rl_loop_200815_1_allonce_3,rl_loop_200815_1_hc_3 --rate_id 5f478d7f60f6bad00ef43ffb
python -m pokeai.ai.generic_move_model.rl_rating_battle 5f37a6829949403bf6af940f rl_loop_200815_1_allonce_4,rl_loop_200815_1_hc_4 --rate_id 5f478d7f60f6bad00ef43ffc
python -m pokeai.ai.generic_move_model.rl_rating_battle 5f37a6829949403bf6af9410 rl_loop_200815_1_allonce_5,rl_loop_200815_1_hc_5 --rate_id 5f478d7f60f6bad00ef43ffd
python -m 

In [15]:
iter_ids = []
for config, rate_id in zip(configs, rate_ids):
    c = copy.deepcopy(config)
    c["rate_id"] = str(rate_id)
    iter_ids.append(c)
with open(os.path.join(file_base_dir, f"iter_ids.json"), "w") as f:
    json.dump(iter_ids, f)